In [7]:
import pandas as pd
import numpy as np

from stump import *

In [8]:
#converting input to the required format

from sklearn.impute import SimpleImputer

train = pd.read_csv('train_classif.csv')
test = pd.read_csv('test_classif.csv')

relevant_features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']

imputer = SimpleImputer(strategy='most_frequent')
train[relevant_features] = imputer.fit_transform(train[relevant_features])
test[relevant_features] = imputer.transform(test[relevant_features])

#encoding
train['Sex'] = train['Sex'].map({'male':0, 'female':1})
test['Sex'] = test['Sex'].map({'male':0, 'female':1})
train['Embarked'] = train['Embarked'].map({'S':0,'C':1,'Q':2})
test['Embarked'] = test['Embarked'].map({'S':0,'C':1,'Q':2})

X = train[relevant_features].to_numpy()
X_test = test[relevant_features].to_numpy()
Y = train['Survived'].to_numpy()

In [9]:
class adaboost_for_classif:
    
    def __init__(self):
        self.stump_list = []
        
    def add_stump(self,X,Y,weights):
        
        o = stump()
        weights = o.best_stump(X,Y,weights,'classif')
        self.stump_list.append(o)
        
        return weights
                
    def build(self, X, Y, n_iter):
        
        # giving equal weight to all datapoints
        weights = [1/X.shape[0] for i in range(X.shape[0])]
        
        for i in range(n_iter):
            weights = self.add_stump(X,Y,weights)
            

In [10]:
o = adaboost_for_classif()
o.build(X,Y,10)
print(o.stump_list)

# test['Survived'] = o.predict(X_test)
# submissions = test[['PassengerId', 'Survived']]

# submissions.to_csv('submission.csv', index=False )

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
[<stump.stump object at 0x00000187E7BCD960>, <stump.stump object at 0x00000187E7BCEFE0>, <stump.stump object at 0x00000187E7BCC160>, <stump.stump object at 0x00000187E7BCF0D0>, <stump.stump object at 0x00000187E7BCD030>, <stump.stump object at 0x00000187E7BCCB80>, <stump.stump object at 0x00000187E7BCF430>, <stump.stump object at 0x00000187E7BCD180>, <stump.stump object at 0x00000187E7BCD270>, <stump.stump object at 0x00000187E7BCD870>]
